In [ ]:
from keras import models
from keras import layers
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(32, 32, 3))
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(512,activation='relu'))
model.add(layers.Dense(1118, activation='sigmoid'))

In [ ]:
import numpy as np
from array import array

def gray_to_rgb(img):
    return  np.squeeze(np.stack((img,) * 3, -1))

def batch_channel_convert(imgs):
    converted=np.empty([86086,32, 32,3])
    for index,img in enumerate(imgs):
        processed=gray_to_rgb(img)
        converted[index]=processed
    return converted

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
from keras import optimizers


X=pd.read_pickle("/kaggle/input/zoom-32-32-1/datas.pickle")
y=pd.read_pickle("/kaggle/input/zoom-32-32-1/labels.pickle")

convertedX=batch_channel_convert(X)
le=LabelEncoder()

y_labels=le.fit_transform(y)


X_train, X_test, y_train, y_test = train_test_split(convertedX, y_labels, train_size=76086, random_state=42)


train_datagen = ImageDataGenerator(    
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(
    x=X_train,
    y=y_train,
    batch_size=1409)


validation_generator = test_datagen.flow(
    x=X_test,y=y_test,
    batch_size=20)

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

history = model.fit_generator(
      train_generator,
      steps_per_epoch=54,
      epochs=100,
      validation_data=validation_generator,
      validation_steps=500,
      verbose=1)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()